In [119]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
import csv
import re

In [123]:
with open('PBskinperc.csv','w') as f1:
    writer=csv.writer(f1, delimiter=',',lineterminator='\n',)

    ctr = 0
    for root,dirs,files in os.walk(".\Playboy", topdown=False):
        for file in files:      
            ctr = ctr+1
            #if ctr >300 and ctr <320 and file.endswith(".jpg"):
            if file.endswith(".jpeg"):
                print ctr
                
                #Establish names
                name = os.path.join("Playboy",file)
                fname = os.path.join("Playboy_Skin",file)
                
                print name

                # Load Data
                img = cv2.imread(name)
                # Convert to HSV
                img2 = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

                # Create the haar cascade
                faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

                # Read the image
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                # Detect faces in the image
                faces = faceCascade.detectMultiScale(
                    gray,
                    scaleFactor=1.1,
                    minNeighbors=5,
                    minSize=(30, 30),
                    flags = cv2.cv.CV_HAAR_SCALE_IMAGE
                )

                print "Found {0} faces!".format(len(faces))

                #Proceed only if just 1 face is identified
                if len(faces) == 1:
                    
                    # Draw a rectangle around the faces
                    for (x, y, w, h) in faces:
                        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

                    #cv2.imshow("Faces found", img)
                    #cv2.waitKey(0)

                    mask = np.zeros(img.shape[:2], np.uint8)

                    a = faces[0][1]
                    b = a + faces[0][3]
                    c = faces[0][0]
                    d = c + faces[0][2]

                    facearea = (b-a)*(d-c)

                    mask[a:b, c:d] = 255
                    #masked_img = cv2.bitwise_and(img,img,mask = mask)

                     # 3D color histogram for hue and saturation
                    nbin = 8
                    hist = cv2.calcHist([img2], [0,1,2],mask,[nbin, nbin, nbin], [0, 180, 0, 256, 0, 256])

                    p = 98
                    #print np.percentile(hist,p)

                    idx = np.where( hist > np.percentile(hist,p) )

                    hl = np.linspace(0,180,nbin+1)[idx[0]]
                    hu = np.linspace(0,180,nbin+1)[idx[0]+1]

                    sl = np.linspace(0,256,nbin+1)[idx[1]]
                    su = np.linspace(0,256,nbin+1)[idx[1]+1]


                    vl = np.linspace(0,256,nbin+1)[idx[2]]
                    vu = np.linspace(0,256,nbin+1)[idx[2]+1]

                    skinMask = np.zeros(img2.shape[:2],dtype=np.uint8)
                    for ii in xrange(len(hl)):
                        lower = np.array([hl[ii], sl[ii], vl[ii]], dtype = "uint8")
                        upper = np.array([hu[ii], min(su[ii],255), min(vu[ii],255)], dtype = "uint8")
                        skinMask = skinMask + cv2.inRange(img2, lower, upper)

                    skinMask[np.where( skinMask !=0 )] = 255

                     # apply a series of erosions and dilations to the mask using an elliptical kernel
                    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
                    skinMask = cv2.erode(skinMask, kernel, iterations = 2)
                    skinMask = cv2.dilate(skinMask, kernel, iterations = 2)

                    # blur the mask to help remove noise, then apply the mask to the frame
                    skinMask = cv2.GaussianBlur(skinMask, (3, 3), 0)
                    skin = cv2.bitwise_and(img, img, mask = skinMask)

                    perc = float(len(np.where( skinMask != 0 )[0]))/float((skinMask.shape[0]*skinMask.shape[1]))
                    perc_nml = float(perc)/float(facearea)
                    print perc, perc_nml

                    # show the skin in the image along with the mask
                    #cv2.imshow("images", np.hstack([img2, skin]))

                    #plt.subplot(131), plt.imshow(img)
                    #plt.subplot(132), plt.imshow(img2)
                    #plt.subplot(133), plt.imshow(skin)

                    #plt.show()

                    #Save File
                    cv2.imwrite(fname, skin)
                else:
                    blank = 255*np.ones(img2.shape[:2],dtype=np.uint8)
                    cv2.imwrite(fname, blank)
                    perc_nml = 0
                 
                #Write results to .csv
                if len(name) > 27:
                    csvname = name[18:25]
                    row = [csvname, perc_nml]
                    writer.writerow(row)
            


1
Playboy\Playboy - 1954-01.jpeg
Found 2 faces!
2
Playboy\Playboy - 1954-02.jpeg
Found 1 faces!
0.0487228464419 9.94343804938e-06
3
Playboy\Playboy - 1954-03.jpeg
Found 0 faces!
4
Playboy\Playboy - 1954-04.jpeg
Found 0 faces!
5
Playboy\Playboy - 1954-05.jpeg
Found 1 faces!
0.106880149813 1.80266739438e-05
6
Playboy\Playboy - 1954-06.jpeg
Found 0 faces!
7
Playboy\Playboy - 1954-07.jpeg
Found 0 faces!
8
Playboy\Playboy - 1954-08.jpeg
Found 0 faces!
9
Playboy\Playboy - 1954-09.jpeg
Found 0 faces!
10
Playboy\Playboy - 1954-10.jpeg
Found 1 faces!
0.0 0.0
11
Playboy\Playboy - 1954-11.jpeg
Found 0 faces!
12
Playboy\Playboy - 1954-12.jpeg
Found 2 faces!
13
Playboy\Playboy - 1954.jpeg
Found 2 faces!
14
Playboy\Playboy - 1955-01.jpeg
Found 0 faces!
15
Playboy\Playboy - 1955-02.jpeg
Found 0 faces!
16
Playboy\Playboy - 1955-03.jpeg
Found 0 faces!
17
Playboy\Playboy - 1955-04.jpeg
Found 1 faces!
0.0561835205993 6.49595567109e-06
18
Playboy\Playboy - 1955-05.jpeg
Found 1 faces!
0.0304794007491 1.219